In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import random
from tqdm import tqdm

In [2]:
class KTHProcessedDataset(Dataset):
    def __init__(self, root_dir, sequence_length, transform=None):
        self.root_dir = root_dir
        self.sequence_length = sequence_length
        self.transform = transform
        self.data = []
        self.class_mapping = {}  # To store class ID to name mapping

        # Traverse through action categories and their subfolders
        for label, category in enumerate(sorted(os.listdir(root_dir))):
            category_path = os.path.join(root_dir, category)
            if not os.path.isdir(category_path):
                continue
            self.class_mapping[label] = category  # Map class ID to category name
            for subfolder in os.listdir(category_path):
                subfolder_path = os.path.join(category_path, subfolder)
                if os.path.isdir(subfolder_path):
                    frames = sorted(os.listdir(subfolder_path))  # Ensure frames are ordered

                    # Check if there are enough frames
                    if len(frames) >= sequence_length:
                        self.data.append((subfolder_path, frames, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        subfolder_path, frames, label = self.data[idx]

        # Select frames sequentially from the start, up to sequence_length
        selected_frames = frames[:self.sequence_length]

        sequence = []
        for frame_file in selected_frames:
            frame_path = os.path.join(subfolder_path, frame_file)
            try:
                # Try to open the image
                img = Image.open(frame_path).convert("L")  # Convert to grayscale
                if self.transform:
                    img = self.transform(img)

                # Flatten the image to a 1D tensor
                img = img.view(-1)  # Flattening the image to size 4096 (64x64)
                sequence.append(img)

            except (IOError, OSError) as e:
                # Log the error and skip the corrupted frame
                print(f"Warning: Skipping corrupted image {frame_path} due to error: {e}")
                return self.__getitem__((idx + 1) % len(self))  # Skip to the next sample

        # Stack frames into a tensor of shape [sequence_length, 4096]
        sequence = torch.stack(sequence, dim=0)
        return sequence, label


In [3]:
# Define augmentations and transformations
transform = transforms.Compose([
    # Spatial augmentations
    # RandomHorizontalFlip(p=0.5),                # Flip frames horizontally with 50% probability
    # RandomRotation(degrees=15),                # Random rotation within ±15 degrees
    transforms.RandomCrop(size=(64, 64), pad_if_needed=True),  # Random crop to 64x64, pad if needed

    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2)], p=0.3),  # Adjust brightness/contrast
    transforms.GaussianBlur(3, sigma=(0.1, 2.0)),  # Random blur
    
    # Conversion and normalization
    transforms.ToTensor(),                                # Convert PIL image to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])           # Normalize to [-1, 1]
])

In [4]:
# Paths and hyperparameters
root_dir = '/home/nfs/inf6/data/datasets/kth_actions/processed'
sequence_length = 45
batch_size = 8
train_ratio = 0.8  # 80% for training, 20% for validation

In [5]:
# Load the full dataset
dataset = KTHProcessedDataset(root_dir=root_dir, sequence_length=sequence_length, transform=transform)

# Calculate train and validation sizes
dataset_size = len(dataset)
train_size = int(train_ratio * dataset_size)
val_size = dataset_size - train_size

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for train and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print(f"Dataset size: {dataset_size}")
print(f"Train set size: {train_size}")
print(f"Validation set size: {val_size}")

Dataset size: 599
Train set size: 479
Validation set size: 120


In [6]:
print("Dataset Classes:")
for class_id, class_name in dataset.class_mapping.items():
    print(f"Class ID: {class_id}, Class Name: {class_name}")

Dataset Classes:
Class ID: 0, Class Name: boxing
Class ID: 1, Class Name: handclapping
Class ID: 2, Class Name: handwaving
Class ID: 3, Class Name: jogging
Class ID: 4, Class Name: running
Class ID: 5, Class Name: walking


In [7]:
import wandb

In [8]:
def train_nn(model, train_loader, criterion, optimizer, device):
    """Train the LSTM model for one epoch"""
    model.train()
    running_loss = 0.0
    correct_labels = 0
    total_labels = 0

    for inputs, labels in train_loader:
        inputs = inputs.view(-1, 45, 1, height, width).to("cuda")
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)  # Get outputs for the full sequence
        
        # Only use the last time step's output for classification
        loss = criterion(outputs, labels)  # Use the last timestep output
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Compute accuracy for the last time step
        _, predicted_labels = torch.max(outputs, dim=1)
        correct_labels += (predicted_labels == labels).sum().item()
        total_labels += labels.size(0)

    avg_loss = running_loss / len(train_loader)
    accuracy = correct_labels / total_labels
    return avg_loss, accuracy

def evaluate_nn(model, eval_loader, criterion, device):
    """Evaluate the LSTM model"""
    model.eval()
    running_loss = 0.0
    correct_labels = 0
    total_labels = 0

    with torch.no_grad():
        for inputs, labels in eval_loader:
            inputs = inputs.view(-1, 45, 1, height, width).to("cuda")
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Get outputs for the full sequence
            
            loss = criterion(outputs, labels)  # Use the last timestep output
            running_loss += loss.item()

            # Compute accuracy for the last time step
            _, predicted_labels = torch.max(outputs, dim=1)
            correct_labels += (predicted_labels == labels).sum().item()
            total_labels += labels.size(0)

    avg_loss = running_loss / len(eval_loader)
    accuracy = correct_labels / total_labels
    return avg_loss, accuracy

def run_training(model, train_loader, eval_loader, criterion, device, num_epochs, learning_rate=0.001, step_size=5, gamma=0.5, project_name='lstm_training'):
    """Train and evaluate the LSTM model for a given number of epochs with W&B logging"""
    # Initialize W&B logging
    wandb.init(project=project_name, config={
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "step_size": step_size,
        "gamma": gamma,
        "optimizer": "Adam",
    },
    )
    config = wandb.config

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)  # Learning rate scheduler
    
    train_losses = []
    eval_losses = []
    train_accuracies = []
    eval_accuracies = []
    learning_rates = []

    for epoch in tqdm(range(num_epochs)):
        # Train for one epoch
        train_loss, train_accuracy = train_nn(model, train_loader, criterion, optimizer, device)
        # Evaluate after each epoch
        eval_loss, eval_accuracy = evaluate_nn(model, eval_loader, criterion, device)

        # Get current learning rate
        current_lr = scheduler.get_last_lr()[0]

        # Record the metrics
        train_losses.append(train_loss)
        eval_losses.append(eval_loss)
        train_accuracies.append(train_accuracy)
        eval_accuracies.append(eval_accuracy)
        learning_rates.append(current_lr)

        # Log metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
            "eval_loss": eval_loss,
            "eval_accuracy": eval_accuracy,
            "learning_rate": current_lr
        })

        # Print epoch statistics
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Eval Loss: {eval_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}")
        print(f"Learning Rate: {current_lr:.6f}")
        
        # Step the learning rate scheduler
        scheduler.step()

    # Finish W&B run
    wandb.finish()

    return model, train_losses, train_accuracies, eval_losses, eval_accuracies, learning_rates

### ConvLSTM

In [9]:
class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.bias = bias

        # Gates: i, f, o, g
        self.conv = nn.Conv2d(
            input_channels + hidden_channels, 
            4 * hidden_channels, 
            kernel_size, 
            padding=self.padding, 
            bias=self.bias
        )

    def forward(self, x, hidden):
        h_prev, c_prev = hidden

        # Concatenate along channel dimension
        combined = torch.cat([x, h_prev], dim=1)
        conv_output = self.conv(combined)

        # Split into gates
        i, f, o, g = torch.split(conv_output, self.hidden_channels, dim=1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        # Update cell state and hidden state
        c_next = f * c_prev + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, height, width):
        h = torch.zeros(batch_size, self.hidden_channels, height, width, device=self.conv.weight.device)
        c = torch.zeros(batch_size, self.hidden_channels, height, width, device=self.conv.weight.device)
        return h, c

In [10]:
class ConvLSTM(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size, num_layers, num_classes, height, width, bias=True):
        super(ConvLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_channels = hidden_channels
        self.height = height
        self.width = width

        # ConvLSTM layers
        self.layers = nn.ModuleList([
            ConvLSTMCell(
                input_channels=input_channels if i == 0 else hidden_channels[i - 1],
                hidden_channels=hidden_channels[i],
                kernel_size=kernel_size,
                bias=bias
            ) for i in range(num_layers)
        ])

        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_channels[-1] * height * width, num_classes)

    def forward(self, x):
        """
        x: [batch_size, sequence_length, channels, height, width]
        Returns logits for classification.
        """
        # print(x.shape)
        batch_size, seq_len, _, height, width = x.size()
        assert height == self.height and width == self.width, "Input size mismatch with initialized height and width."

        # Initialize hidden states for all layers
        hidden_states = [layer.init_hidden(batch_size, height, width) for layer in self.layers]

        # Process sequence through ConvLSTM layers
        for t in range(seq_len):
            current_input = x[:, t]
            for l, layer in enumerate(self.layers):
                hidden_states[l] = layer(current_input, hidden_states[l])
                current_input = hidden_states[l][0]  # Use the hidden state (h_t)

        # Last layer's hidden state at the last time step
        last_hidden_state = hidden_states[-1][0]  # Shape: [batch_size, hidden_channels[-1], height, width]

        # Flatten and classify
        last_hidden_state_flat = last_hidden_state.view(batch_size, -1)  # Flatten: [batch_size, hidden_channels[-1] * height * width]
        logits = self.fc(last_hidden_state_flat)  # [batch_size, num_classes]
        return logits

In [11]:
# Initialize ConvLSTM
input_channels = 1  # Grayscale images
hidden_channels = [64, 128]  # Two layers with 64 and 128 hidden channels
kernel_size = 3
num_layers = len(hidden_channels)
num_classes=6
height=64
width=64

model = ConvLSTM(
    input_channels=input_channels,
    hidden_channels=hidden_channels,
    kernel_size=kernel_size,
    num_layers=num_layers,
    num_classes=num_classes,
    height=height,
    width=width
).to("cuda")

In [12]:
model

ConvLSTM(
  (layers): ModuleList(
    (0): ConvLSTMCell(
      (conv): Conv2d(65, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvLSTMCell(
      (conv): Conv2d(192, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (fc): Linear(in_features=524288, out_features=6, bias=True)
)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyperparameters
num_epochs = 40
learning_rate = 0.001
step_size = 5
gamma = 0.5

# Run training
model, train_losses, train_accuracies, eval_losses, eval_accuracies, learning_rates = run_training(
    model, train_loader, val_loader, criterion, device,
    num_epochs, learning_rate, step_size, gamma, 'convlstm_training'
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ahmadjaved97. Use `wandb login --relogin` to force relogin


  2%|█▍                                                      | 1/40 [00:43<28:29, 43.83s/it]

Epoch 1/40
Train Loss: 1.9241, Train Accuracy: 0.2109
Eval Loss: 1.5918, Eval Accuracy: 0.2917
Learning Rate: 0.001000


  5%|██▊                                                     | 2/40 [01:27<27:46, 43.86s/it]

Epoch 2/40
Train Loss: 1.6205, Train Accuracy: 0.3361
Eval Loss: 1.5339, Eval Accuracy: 0.3500
Learning Rate: 0.001000


In [17]:
# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_idx, (sequences, labels) in enumerate(train_loader):
        # Reshape input to [batch_size, sequence_length, channels, height, width]
        sequences = sequences.view(-1, 45, 1, height, width).to("cuda")
        labels = labels.to("cuda")

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(sequences)  # Logits
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        # Backward pass and update weights
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(train_loader):.4f}")

Epoch [1/10], Loss: 1.6820
Epoch [2/10], Loss: 1.4542
Epoch [3/10], Loss: 1.2875
Epoch [4/10], Loss: 1.0374
Epoch [5/10], Loss: 0.7503


KeyboardInterrupt: 